In [ ]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime

import pandas as pd
from tqdm import tqdm

from moment.common import PATHS

In [ ]:
finetuning_mode = 'linear-probing' #'end-to-end'
experiment_name = "supervised_forecasting" #"supervised_forecasting/N-BEATS"
dataset_names = ['electricity', 'ETTh1', 'ETTh2', 'ETTm1', 'ETTm2', 'national_illness', 'traffic', 'weather']
forecast_horizons = ['96', '720']
forecast_horizons_illness = ['24', '60']

results_path = os.path.join(PATHS.RESULTS_DIR, experiment_name)
print("Results path:", results_path)

In [ ]:
incomplete_experiments = []
results = []

for dataset_name in dataset_names:
    horizons = None
    horizons = forecast_horizons_illness if dataset_name == 'national_illness' else forecast_horizons
    
    for horizon in horizons:
        results_path_complete = os.path.join(
            results_path, dataset_name, finetuning_mode, horizon)
        
        if not os.path.exists(results_path_complete) or len(os.listdir(results_path_complete)) == 0:
            incomplete_experiments.append((dataset_name, horizon))
            continue
        
        filenames = os.listdir(results_path_complete)
        optim_steps = [f.split('_')[-1][:-4] for f in filenames]
        optim_steps = [int(s) for s in optim_steps]
        final_optim_step = max(optim_steps)
        best_result_path = os.path.join(results_path_complete, f"results_long-horizon-forecasting_{final_optim_step}.csv")

        df = pd.read_csv(best_result_path, index_col=0).T
        df['Dataset Name'] = dataset_name
        df['Forecast Horizon'] = horizon
        last_modified_time = int(os.path.getmtime(best_result_path))
        last_modified_time = datetime.utcfromtimestamp(last_modified_time).strftime('%d-%m-%Y %H:%M')
        df['Timestamp'] = last_modified_time

        results.append(df)
        
results = pd.concat(results, axis=0).reset_index(drop=True)
results = results[[
    'Dataset Name', 'Forecast Horizon', 'MSE', 
    'MAE', 'MAPE', 'sMAPE', 'RMSE', 'Opt. steps', 
    'Model name', 'ID', 'Timestamp']]

results.to_csv("../../assets/results/finetuning/forecasting.csv", index=False)

In [ ]:
print(f"Results: {results.shape}" )
results

In [ ]:
import argparse

from moment.data.classification_datasets import get_classification_datasets
from moment.data.dataloader import get_timeseries_dataloader
from moment.utils.config import Config
from moment.utils.utils import parse_config

In [ ]:
config_path = "../../configs/classification/unsupervised_representation_learning.yaml"
defualt_config_path = "../../configs/default.yaml"

config = Config(config_file_path=config_path, default_config_file_path=defualt_config_path).parse()
args = parse_config(config)

In [ ]:
args.full_file_path_and_name = '/TimeseriesDatasets/classification/UCR/GestureMidAirD2/GestureMidAirD2_TEST.ts'

In [ ]:
train_dataloader, test_dataloader, val_dataloader =\
              get_dataloaders(args)